In [ ]:
# This comes from https://github.com/harterrt/cookiecutter-python-etl/
# Thanks Harter!

In [ ]:
import boto3
import botocore
import os

from io import BytesIO
from gzip import GzipFile

In [ ]:
repo_dir = "probe-scraper"
output_dir = "/home/hadoop/analyses/probe_data"
cache_dir = "/home/hadoop/analyses/probe_cache"
repo_https_url = "https://github.com/mozilla/probe-scraper"

S3_PUBLIC_BUCKET = "telemetry-public-analysis-2"
S3_DATA_PATH = "probe-scraper/data-rest/"

In [ ]:
!rm -rf $repo_dir

In [ ]:
!rm -rf $output_dir

In [ ]:
!rm -rf $cache_dir

In [ ]:
!git config --global user.email "gfritzsche@mozilla.com" && \
git config --global user.name "Georg Fritzsche"

In [ ]:
!git clone $repo_https_url $repo_dir

In [ ]:
!cd $repo_dir && git pull origin master && python setup.py bdist_egg

In [ ]:
!mkdir $output_dir && mkdir $cache_dir

In [ ]:
!cd $repo_dir && pip install -r requirements.txt && python probe_scraper/runner.py --outdir $output_dir --tempdir $cache_dir

## Upload the output to S3.

In [ ]:
# Get access to the S3 connect API.
client = boto3.client('s3', 'us-west-2')

In [ ]:
def gzip_compress(source_file):
    """ Apply GZIP compression to the content of the provided file.

    :param source_file: the absolute path of the file to compress.
    :return: The gzip compressed content of the input file.
    """
    with open(source_file) as fi:
        text_body = fi.read().decode("utf-8")

    gz_body = BytesIO()
    gz = GzipFile(None, 'wb', 9, gz_body)
    gz.write(text_body.encode('utf-8'))
    gz.close()
    
    return gz_body.getvalue()

In [ ]:
for path, subdirs, files in os.walk(output_dir):
    relative_path = os.path.relpath(path, output_dir)
    # GZIP-compress the files, then copy them to S3. Allow caching for 8 hours.
    for file_name in files:
        source_path = os.path.join(path, file_name)
        key_path = os.path.join(S3_DATA_PATH, relative_path, file_name)
        print "uploading " + file_name + " to s3: " + key_path
        client.put_object(ACL='public-read', Bucket=S3_PUBLIC_BUCKET,
                          Key=key_path, Body=gzip_compress(source_path),
                          ContentEncoding='gzip', CacheControl='max-age=28800',
                          ContentType='application/json')